# Preprocessing

In [ ]:
import pandas as pd
import csv
import random

## 1. Categories

In [ ]:
df_cat = pd.read_csv('category/wikipedia_category.tsv', sep='\t')
df_cat

In [ ]:
df_cat.rename(columns={'cat_id': 'category_id',
                       'cat_title': 'title',
                       'cat_pages': 'pages',
                       'cat_subcats': 'subcats',
                       'cat_files': 'files'},
              inplace=True)
df_cat

In [ ]:
cat_names = df_cat[['category_id', 'category_title']].drop_duplicates()
cat_names

In [ ]:
df_cat.to_csv('data/category.tsv', sep='\t', index=False)

## 2. Category links

In [ ]:
df_cl = pd.read_csv('categorylinks/wikipedia_page_category.tsv', sep='\t')
df_cl

In [ ]:
df_cl = pd.merge(df_cl, cat_names, how='inner', left_on='cl_to', right_on='category_title')
df_cl = df_cl[['cl_from', 'category_id', 'cl_type']].drop_duplicates()
df_cl.rename(columns={'cl_from': 'page_id',
                   'cl_type': 'type'},
         inplace=True)
df_cl

In [ ]:
df_cl.to_csv('data/page_category.tsv', sep='\t', index=False)

## 3. References

In [ ]:
df_ref = pd.read_csv('references/wp_ref.tsv', sep='\t')
df_ref.rename(columns={'id': 'page_id',
                       'r_id': 'identifier',
                       'type': 'identifier_type'},
             inplace=True)
df_ref

In [ ]:
df_ref['identifier_type'] = df_ref['identifier_type'].str.lower()
df_ref['identifier'] = df_ref['identifier'].str.lower()
df_pub = df_ref[['pub_id', 'identifier_type', 'identifier']].drop_duplicates()
df_pub

In [ ]:
df_pub = df_pub.pivot(index='pub_id', columns='identifier_type', values='identifier')
df_pub.fillna('', inplace=True)
df_pub

In [ ]:
df_pub['id'] = (df_pub['arxiv'].astype(str) + '|' +
                df_pub['asin'].astype(str) +  '|' +
                df_pub['bibcode'].astype(str) +  '|' +
                df_pub['doi'].astype(str) + '|' +
                df_pub['isbn'].astype(str) + '|' +
                df_pub['ismn'].astype(str) + '|' +
                df_pub['jfm'].astype(str) + '|' +
                df_pub['jstor'].astype(str) + '|' +
                df_pub['lccn'].astype(str) + '|' +
                df_pub['mr'].astype(str) + '|' +
                df_pub['oclc'].astype(str) + '|' +
                df_pub['ol'].astype(str) + '|' +
                df_pub['osti'].astype(str) + '|' +
                df_pub['pmc'].astype(str) + '|' +
                df_pub['pmid'].astype(str) + '|' +
                df_pub['rfc'].astype(str) + '|' +
                df_pub['ssrn'].astype(str) + '|' +
                df_pub['url'].astype(str) + '|' +
                df_pub['usenetid'].astype(str) + '|' +
                df_pub['zbl'].astype(str))
df_pub

In [ ]:
df_pub.reset_index(inplace=True)
df_pub

In [ ]:
len(set(df_pub['id'].tolist()))

In [ ]:
df_pub[['id']]

In [ ]:
df_ref = pd.merge(df_ref, df_pub[['pub_id', 'id']], left_on='pub_id', right_on='pub_id')
df_ref

In [ ]:
df_pub.drop('pub_id', axis=1, inplace=True)
df_pub = df_pub.drop_duplicates()
df_pub

In [ ]:
df_pub['pub_id'] = range(1,df_pub.shape[0]+1)
df_pub

In [ ]:
df_ref = pd.merge(df_ref[['page_id', 'id', 'identifier_type', 'identifier']], df_pub[['pub_id', 'id']], left_on='id', right_on='id')
df_ref = df_ref[['page_id', 'pub_id']].drop_duplicates()
df_ref

In [ ]:
df_ref.to_csv('data/page_pub.tsv', sep='\t', index=False)

In [ ]:
df_pub = df_pub[['pub_id', 'arxiv', 'asin', 'bibcode', 'doi', 'isbn','ismn', 'jfm', 'jstor', 'lccn', 'mr', 'oclc', 'ol', 'osti', 'pmc', 'pmid', 'rfc', 'ssrn', 'url', 'usenetid', 'zbl']]
df_pub = df_pub.drop_duplicates()
df_pub

In [ ]:
df_pub.to_csv('data/pub.tsv', sep='\t', index=False)

## 4. Pagelinks

In [ ]:
df_pl = pd.read_csv('pagelinks/wp_pagelinks.tsv', sep='\t')
df_pl

### 4.1. Pages

Reduce Wikipedia pages to articles (namespace=0).

In [ ]:
df_pages = pd.read_csv('page/page.tsv', sep='\t')
df_pages = df_pages[df_pages['page_namespace']==0]
df_pages = df_pages[['page_id', 'page_title']]
df_pages

In [ ]:
df_pl = pd.merge(df_pl, df_pages, how='inner', left_on='pl_title', right_on='page_title')
df_pl

In [ ]:
a = df_pages['page_title'].tolist()
b = df_pages['page_id'].tolist()

In [ ]:
df_pl = df_pl[df_pl['pl_title'].isin(a)]
df_pl

In [ ]:
df_pl = df_pl[df_pl['pl_from'].isin(b)]
df_pl

In [ ]:
c = df_pl['pl_title'][0:5]
c

In [ ]:
replacement_map = {i1: i2 for i1, i2 in zip(a, b)}
c.map(replacement_map)

In [ ]:
df_pl['pl_title'] = df_pl['pl_title'].map(replacement_map)
df_pl

In [ ]:
df_pl.rename(columns={'pl_from': 'page_id',
                      'pl_title': 'linked_page_id'},
            inplace=True)
df_pl

In [ ]:
df_pl.to_csv('data/page_link.tsv', sep='\t', index=False)

## 5. Pages

Merging with other datasets (views and edits).

In [ ]:
df_pages = pd.read_csv('page/page.tsv', sep='\t')
df_pages

### 5.1. Views

In [ ]:
df_pv = pd.read_csv('pageviews/page_views.tsv', sep='\t', quoting=csv.QUOTE_NONE)
df_pv.rename(columns={'title':'page_title',
                      'daily_total': 'views'},
            inplace=True)
df_pv

In [ ]:
df_pv[df_pv['page_title']=='Magnolia_(film)']

In [ ]:
df_pv[~ df_pv['page_title'].isin(df_pages['page_title'].tolist())].head(50)

In [ ]:
df_pages[df_pages['page_title']=='"Weird_Al"_Yankovic_songs']

In [ ]:
df_pages = pd.merge(df_pages, df_pv, how='left', on='page_title')
df_pages

In [ ]:
df_pages['views'] = df_pages['views'].astype('Int64')
df_pages

In [ ]:
df_pages['page_links_updated'] = df_pages['page_links_updated'].astype('Int64')
df_pages

### 5.2. Edits

In [ ]:
df_pr = pd.read_csv('edits/page_edits.tsv', sep='\t')
df_pr

In [ ]:
df_pr[~ df_pr['page_id'].isin(df_pages['page_id'].tolist())]

In [ ]:
df_pages = pd.merge(df_pages, df_pr, how='left', on='page_id')
df_pages

In [ ]:
df_pages['count'] = df_pages['count'].astype('Int64')
df_pages['contributor'] = df_pages['contributor'].astype('Int64')
df_pages

In [ ]:
df_pages.rename(columns={'page_namespace': 'namespace',
                         'page_title': 'title',
                         'page_restrictions': 'restrictions',
                         'page_is_redirect': 'is_redirect',
                         'page_is_new': 'is_new',
                         'page_touched': 'touched',
                         'page_links_updated': 'links_updated',
                         'page_latest': 'latest',
                         'page_len': 'len',
                         'page_content_model': 'content_model',
                         'count': 'edits',
                         'time': 'creation',
                         'contributor': 'editors'},
               inplace=True)
df_pages

In [ ]:
df_pages.to_csv('data/page.tsv', sep='\t', index=False, quoting=csv.QUOTE_NONE)

## 6. Page_props

In [ ]:
df_pp = pd.read_csv('page_props/wp_page_props.tsv', sep='\t')
df_pp.rename(columns={'pp_page':'page_id',
                      'pp_propname': 'property_name',
                      'pp_value': 'property'},
            inplace=True)
df_pp

In [ ]:
df_pp.to_csv('data/page_property.tsv', sep='\t', index=False)

## 7. Page URL

In [ ]:
df_purl = pd.read_csv('url_normalization/page_url.tsv', sep='\t')
df_purl

In [ ]:
df_purl.rename(columns={'ref': 'in_reference'},
               inplace=True)
df_purl

In [ ]:
df_purl.to_csv('data/page_url.tsv', sep='\t', index=False, quoting=csv.QUOTE_NONE)